
# Diffie Hellman
With key exchange, Bob and Alice end up with the same secret key, even though Eve is listening to their communications:

<img src='graphics/g_key_02.png' width="800px">

The Diffie-Hellman method allows Bob and Alice to pass public values to each other and then derive the same key:

<img src='graphics/g_key_01.png' width="800px">

For this Bob generates a secret value (b) and Alice generates a secret value (a). Bob passes g^b mod p, and Alice passes g^a mod p.


> Bob and Alice have agreed on the values:

g=2879, N= 9929

Bob Select x=6, Alice selects y=9

Now calculate (using a calculator or Python:

Alice’s A value (g^x mod N):

Bob’s B value (g^y mod N):

Now they exchange the values. Next calculate the shared key:

Alice’s value (B^x mod N):

Bobs’s value (A^y mod N):

> Do they match? Yes/No

We can use Python to implement a simple example with a given base generator (g) and prime number (p):

In [6]:
# 04_01.py
# https://asecuritysite.com/keyexchange/diffie_py
import random
import hashlib

g=5
p=1001

a=random.randint(1, p)

b=random.randint(1,p)

A = (g**a) % p
B = (g**b) % p

print('g: ',g,' (a shared value), n: ',p, ' (a prime number)')

print('\nAlice calculates:')
print('a (Alice random): ',a)
print('Alice value (A): ',A,' (g^a) mod p')

print('\nBob calculates:')
print('b (Bob random): ',b)
print('Bob value (B): ',B,' (g^b) mod p')

print('\nAlice calculates:')
keyA=(B**a) % p
print('Key: ',keyA,' (B^a) mod p')
print('Key: ',hashlib.sha256(str(keyA).encode()).hexdigest())

print('\nBob calculates:')
keyB=(A**b) % p
print('Key: ',keyB,' (A^b) mod p')
print('Key: ',hashlib.sha256(str(keyB).encode()).hexdigest())

g:  5  (a shared value), n:  1001  (a prime number)

Alice calculates:
a (Alice random):  410
Alice value (A):  298  (g^a) mod p

Bob calculates:
b (Bob random):  105
Bob value (B):  265  (g^b) mod p

Alice calculates:
Key:  155  (B^a) mod p
Key:  210e3b160c355818509425b9d9e9fd3ea2e287f2c43a13e5be8817140db0b9e6

Bob calculates:
Key:  155  (A^b) mod p
Key:  210e3b160c355818509425b9d9e9fd3ea2e287f2c43a13e5be8817140db0b9e6


> Run the code and verify that Bob and Alice compute the same shared secret.

> The (g**b) % p operation can be computationally intensive. Replace this operation with pow(g,b,p), and verify that the program still works.

> Use a prime number of 997, and verify the operation of the code.

> Use a prime number of 2^19-1, and verify the operation of the code.

## Selecting generator for discrete logs
With a discrete log method, we have the form of:

Y=g^x (mod p)

For the base, we must select a base generator that produces all the possible values of Y for x, with a given prime number of p.




In [1]:
# 04_02.py
# https://asecuritysite.com/dh/pickg2
p=11

def getG(p):

  for x in range (1,p):
	  rand = x
	  exp=1
	  next = rand % p

	  while (next != 1 ):
		  next = (next*rand) % p
		  exp = exp+1


	  if (exp==p-1):
		  print (rand,end=' ')

print ("p: ",p)
print ("Values for g^x mod p: ")
print (getG(p))

p:  11
Values for g^x mod p: 
2 6 7 8 None


> For a prime number of p=11, what are the possible base generator values?
> For a prime number of p=97, what are the possible base generator values?
> If we pick a value of p=11, and take a value of g=5, show that we do not output all the possible values for Y=g^x (mod p).
> If we pick a value of p=11, and take a value of g=6, show that we do output all the possible values for Y=g^x (mod p).

# Elliptic Curve Diffie Hellman (ECDH)
The most popular key exchange method is ECDH (Elliptic Curve Diffie Hellman). This uses an elliptic curve method, of which the most popular curve is P256 (SECP256R1). 

<img src='graphics/g_key_12.png' width="800px">

The following is the some code:

In [7]:
# 04_03.py
# https://asecuritysite.com/hazmat/hashnew13
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
import binascii
import sys

Bob_private_key = ec.generate_private_key(ec.SECP384R1(),default_backend())

Alice_private_key = ec.generate_private_key(ec.SECP384R1(),default_backend())
size=32 # 256 bit key



Bob_private_key = ec.generate_private_key(ec.SECP256R1(),default_backend())
Alice_private_key = ec.generate_private_key(ec.SECP256R1(),default_backend())  

Bob_shared_key = Bob_private_key.exchange(ec.ECDH(), Alice_private_key.public_key())

Bob_derived_key = HKDF(algorithm=hashes.SHA256(),length=size,salt=None,info=b'',backend=default_backend()).derive(Bob_shared_key)

Alice_shared_key = Alice_private_key.exchange(ec.ECDH(), Bob_private_key.public_key())

Alice_derived_key = HKDF(algorithm=hashes.SHA256(),length=size,salt=None,info=b'',backend=default_backend()).derive(Alice_shared_key)

print ("Name of curve: ",Bob_private_key.public_key().curve.name)
print (f"Generated key size: {size} bytes ({size*8} bits)")

vals = Bob_private_key.private_numbers()
print (f"\nBob private key value: {vals.private_value}")
vals=Bob_private_key.public_key()
enc_point=binascii.b2a_hex(vals.public_bytes(encoding=serialization.Encoding.DER,format=serialization.PublicFormat.SubjectPublicKeyInfo)).decode()
print("Bob's public key: ",enc_point)

vals = Alice_private_key.private_numbers()
print (f"\nAlice private key value: {vals.private_value}")
vals=Alice_private_key.public_key()
enc_point=binascii.b2a_hex(vals.public_bytes(encoding=serialization.Encoding.DER,format=serialization.PublicFormat.SubjectPublicKeyInfo)).decode()
print("Alice's public key: ",enc_point)

print ("\nBob's derived key: ",binascii.b2a_hex(Bob_derived_key).decode())
print("Alice's derived key: ",binascii.b2a_hex(Alice_derived_key).decode())

Name of curve:  secp256r1
Generated key size: 32 bytes (256 bits)

Bob private key value: 115386775143760985994606199743124500698563324842662591622007624940178885352350
Bob's public key:  3059301306072a8648ce3d020106082a8648ce3d0301070342000453c9db700d76051a285ac6ecf2a27bc82056faca157b52d9744d1f781577e2a8bd0011523ad95fc5012b2c1d5bbae06a51f4a64e34e5397e4c282fe595bf8037

Alice private key value: 73556695653589803082300318169887127685187875212044508782243551530526730975658
Alice's public key:  3059301306072a8648ce3d020106082a8648ce3d03010703420004aa8ac00f7a33ab033eb047d9fe2892a9da3ad8381b8bd1f4e0007f89b13d5b6a243a3ec52af2c48903cdf662c6cf79acc05292bce510f4fcb11071cb6be338f2

Bob's derived key:  c1c76e98cb3ef84986fa5395b8da34bff86c180005ff3de1ea088807542ad517
Alice's derived key:  c1c76e98cb3ef84986fa5395b8da34bff86c180005ff3de1ea088807542ad517


> The program uses SECP256R1, and which is also known as the NIST P-256 curve. The Bitcoin curve is SECP256k1. Modify the program so that it uses this curve. Verify that it still works of the shared key.

> An enhanced curve is SECP521R1. Modify the program so that it uses this curve. Verify that it still works of the shared key.

> The Brainpool curves have some advantages over SECP2561. Modify the program so that it uses the P256R1 curve, and verify that it still works for the shared key.

> The program creates a 256-bit key. Modify the program so that it generated a 128-bit key.

> Modify the program so that is displays the shared key in a Base64 format.


## DH Parameters 
In the DH key generation we have two base parameters: g and p. We can use OpenSSL to generate these parameters. Such as:

>> Generate 768-bit Diffie-Hellman parameters:

openssl dhparam -out dhparams.pem -text 768 

>> View your key exchange parameters with:

cat dhparams.pem	

>> What is the value of g:

>> How many bits does the prime number have?

>> How long does it take to produce the parameters for 1,024 bits (Group 2)?

>> How long does it take to produce the parameters for 1536 bits (Group 5)?

>> How would we change the g value?

## EC Paramters
With elliptic curves, we can use OpenSSL to generate a curve:

>> Let’s look at the Elliptic curves we can create:

openssl ecparam -list_curves

>> We can create our elliptic parameter file with:

openssl ecparam -name secp256k1 -out secp256k1.pem

>> Now view the details with:

openssl ecparam -in secp256k1.pem -text -param_enc explicit -noout

>> What are the details of the key?

>> Now we can create our key pair:

openssl ecparam -in secp256k1.pem -genkey -noout -out mykey.pem

>> Name three 160-bit curves: